<a href="https://colab.research.google.com/github/semishen/Chinese-NLP-Practices/blob/main/09_SimpleTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

In [ ]:
# load data

import numpy as np
import pandas as pd

url = 'https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/google_comments.xlsx'
google = pd.read_excel(url, index_col=0)
print(google.shape)
google.head()

(500, 3)


,username,score,comment
0,倪依芙,5,起初找不到入口，還以為沒座位，但是原來入口在旁邊樓梯～座位區不大，但是料理很好吃～肉圓的醬微...
1,Mimi龍,4,聞名已久，今天特別來現場試試，目前改成點餐機點餐， ⋯⋯更多聞名已久，今天特別來現場試試，目...
2,林昌逸,1,非常生氣服務員的態度😠服務很糟糕，麵都還沒吃完，一起身就把麵收走，擺明趕客人😡😡😡
3,林幸蓁,5,內用與外帶有不同的點餐方式內用須至店內使用機台點菜、輸入桌號並付款外帶好像只需在入口處點餐即...
4,台灣TAXI市區叫車禮車包車約拍找小鄭,5,乾淨衛生，廁所很大在地下室，有免費飲料可以喝，拉麵愛好吃，麻辣豆腐不錯、肉粽不錯，目前吃過很...


In [ ]:
# convert score to status

def score_to_status(x):
  assert isinstance(x, int), 'input is not Int'
  assert (x >= 0) and (x <= 5), 'x is out of range 0-5'

  if x > 3:
    return 'good'
  elif x == 3:
    return 'soso'
  else :
    return 'bad'

In [ ]:
google['status'] = google['score'].map(score_to_status)
print(google['status'].value_counts())

good    239
bad     201
soso     60
Name: status, dtype: int64


In [ ]:
google_good_bad = google[google['status'].isin(['good','bad'])]
print(google_good_bad.shape)

(440, 4)


In [ ]:
# rebuild df as test, label(int) format
df = google_good_bad.drop(['username','score'],axis=1)
df['status'] = df['status'].replace({'good':1, 'bad': 0})
df.head()

,comment,status
0,起初找不到入口，還以為沒座位，但是原來入口在旁邊樓梯～座位區不大，但是料理很好吃～肉圓的醬微...,1
1,聞名已久，今天特別來現場試試，目前改成點餐機點餐， ⋯⋯更多聞名已久，今天特別來現場試試，目...,1
2,非常生氣服務員的態度😠服務很糟糕，麵都還沒吃完，一起身就把麵收走，擺明趕客人😡😡😡,0
3,內用與外帶有不同的點餐方式內用須至店內使用機台點菜、輸入桌號並付款外帶好像只需在入口處點餐即...,1
4,乾淨衛生，廁所很大在地下室，有免費飲料可以喝，拉麵愛好吃，麻辣豆腐不錯、肉粽不錯，目前吃過很...,1


In [ ]:
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df, test_size = 0.2, random_state = 42)
print(train_df.shape)
print(eval_df.shape)

(352, 2)
(88, 2)


# Run GPU

In [ ]:
! nvidia-smi

Sun Sep  6 11:01:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Transformers and SimpleTransformers

In [ ]:
# rebuild environment for BERT
! pip install transformers

In [ ]:
# should install simpletransformers after transformers installed
! pip install simpletransformers

## bert-base-chinese as baseline

In [ ]:
# use bert-base-chinese to build a model
from simpletransformers.classification import ClassificationModel as CM

model = CM('bert', 'bert-base-chinese')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificati

In [ ]:
model.train_model(train_df, args = {'overwrite_output_dir': True})

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:288: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
result, model_output, wrong_predictions = model.eval_model(eval_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:760: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [ ]:
result

{'eval_loss': 0.20669114115563306,
 'fn': 6,
 'fp': 0,
 'mcc': 0.8722783759886469,
 'tn': 40,
 'tp': 42}

In [ ]:
for wrong in wrong_predictions:
  if wrong.label == 0:
    true_label = 'bad'
    predict_label = 'good'
  else:
    true_label = 'good'
    predict_label = 'bad'

  print(f'true: {true_label} | predict: {predict_label} | {wrong.text_a}')

true: good | predict: bad | 意麵瀝得夠乾調味很棒不會吃完兩個小時後還有死鹹味在嘴裡 ⋯⋯更多意麵瀝得夠乾調味很棒不會吃完兩個小時後還有死鹹味在嘴裡不喜歡臭豆腐的我卻很喜歡這家的豆腐因為臭味不明顯卻辛辣鹹香會附上泡菜跟香菜中和味道鴨血也很入味點綜合的話就可以一次吃到豆腐跟鴨血 還會有豆芽跟香菇很多人抱怨環境髒跟店員態度差我是覺得沒多乾淨 但台灣的小吃店其實都差不多啊哈哈哈哈這個價位的餐廳可以支付的薪水店員也沒行有餘力去微笑沒太多時間打掃的清潔溜溜吧
true: good | predict: bad | 可惜了地點優越，應該衛生要做好，以照顧更多大眾，筷子有味道，湯喝下肚也有種味道。。最近有進步了，改成4顆星
true: good | predict: bad | 東西好吃又不貴，唯有的缺點就是很難停車
true: good | predict: bad | 招牌意麵敲好粗！強！But... ⋯⋯更多招牌意麵敲好粗！強！But... 肉圓... 你怎麼了... 怎麼那麼大一顆都是粉... 料哪去了？？？（敲碗ing...）
true: good | predict: bad | 這裡24小時營業，交通很方便，也有配合外送熊貓&Uber,餐點有些很好吃但有些就……所以都只會固定點那幾樣，服務人員大多都很好，就唯獨一位做很久的女店員，每次點餐都愛理不理，要不就是臉很臭，要不就是很不屑，有看到那位女店員在的話，就想轉移陣地了~希望老闆娘能正視一下這個問題，不然很多喜愛玉欣珍的老主顧會因此跑光光的~竹筍湯很好喝，也可以加湯，還有免費的金桔檸檬喝到飽，也很解膩，好喝~
true: good | predict: bad | 真假啦 裡面有個服務生陳韋什麼的 我東西放在那忘了他還特別坐來基隆還我～～


In [ ]:
# compute acc

def compute_acc(result):
  return (result['tp']+result['tn'])/(result['tp']+result['tn']+result['fp']+result['fn'])

acc = compute_acc(result)
print('acc: ', acc)

acc:  0.9318181818181818


# BERT-WWM

In [ ]:
# use BERT WWM
model_WWM = CM('bert', 'hfl/chinese-bert-wwm')

Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint

In [ ]:
model_WWM.train_model(train_df, args = {'overwrite_output_dir': True})

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:288: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
result, model_output, wrong_predictions = model_WWM.eval_model(eval_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:760: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [ ]:
acc = compute_acc(result)
print('acc: ', acc)

acc:  0.9431818181818182


In [ ]:
for wrong in wrong_predictions:
  if wrong.label == 0:
    true_label = 'bad'
    predict_label = 'good'
  else:
    true_label = 'good'
    predict_label = 'bad'

  print(f'true: {true_label} | predict: {predict_label} | {wrong.text_a}')

true: bad | predict: good | 自動點餐機得等一輩子。
true: good | predict: bad | alan哥服務真的好到沒話說  不吃會後悔！
true: good | predict: bad | 招牌意麵敲好粗！強！But... ⋯⋯更多招牌意麵敲好粗！強！But... 肉圓... 你怎麼了... 怎麼那麼大一顆都是粉... 料哪去了？？？（敲碗ing...）
true: good | predict: bad | 這裡24小時營業，交通很方便，也有配合外送熊貓&Uber,餐點有些很好吃但有些就……所以都只會固定點那幾樣，服務人員大多都很好，就唯獨一位做很久的女店員，每次點餐都愛理不理，要不就是臉很臭，要不就是很不屑，有看到那位女店員在的話，就想轉移陣地了~希望老闆娘能正視一下這個問題，不然很多喜愛玉欣珍的老主顧會因此跑光光的~竹筍湯很好喝，也可以加湯，還有免費的金桔檸檬喝到飽，也很解膩，好喝~
true: good | predict: bad | 真假啦 裡面有個服務生陳韋什麼的 我東西放在那忘了他還特別坐來基隆還我～～


# OpenCC
- t2s.json : traditional Chinese to simplified Chinese
- s2t.json : simplified Chinese to traditional Chinese


In [ ]:
# convert text from traditional Chinese to simplified Chinese by openCC

! pip install opencc

In [ ]:
import opencc
t2s = opencc.OpenCC('t2s.json')
df_simplified = df
df_simplified['comment'] = df_simplified['comment'].map(t2s.convert)
df_simplified.head()

,comment,status
0,起初找不到入口，还以为没座位，但是原来入口在旁边楼梯～座位区不大，但是料理很好吃～肉圆的酱微...,1
1,闻名已久，今天特别来现场试试，目前改成点餐机点餐， ⋯⋯更多闻名已久，今天特别来现场试试，目...,1
2,非常生气服务员的态度😠服务很糟糕，面都还没吃完，一起身就把面收走，摆明赶客人😡😡😡,0
3,内用与外带有不同的点餐方式内用须至店内使用机台点菜、输入桌号并付款外带好像只需在入口处点餐即...,1
4,干净卫生，厕所很大在地下室，有免费饮料可以喝，拉面爱好吃，麻辣豆腐不错、肉粽不错，目前吃过很...,1


# BERT-WWM with simplified Chinese texts

In [ ]:
train_df_simplified, eval_df_simplified = train_test_split(df_simplified, test_size = 0.2, random_state = 42)
model_args = {
    "num_train_epochs": 5
}
model_WWM = CM('bert', 'hfl/chinese-bert-wwm', args=model_args)
model_WWM.train_model(train_df_simplified, args = {'overwrite_output_dir': True})
result, model_output, wrong_predictions = model_WWM.eval_model(eval_df_simplified)

Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:760: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [ ]:
acc = compute_acc(result)
print('acc: ', acc)
for wrong in wrong_predictions:
  if wrong.label == 0:
    true_label = 'bad'
    predict_label = 'good'
  else:
    true_label = 'good'
    predict_label = 'bad'

  print(f'true: {true_label} | predict: {predict_label} | {wrong.text_a}')

acc:  0.9659090909090909
true: bad | predict: good | 自动点餐机得等一辈子。
true: good | predict: bad | 招牌意面敲好粗！强！But... ⋯⋯更多招牌意面敲好粗！强！But... 肉圆... 你怎么了... 怎么那么大一颗都是粉... 料哪去了？？？（敲碗ing...）
true: good | predict: bad | 真假啦 里面有个服务生陈韦什么的 我东西放在那忘了他还特别坐来基隆还我～～
